In [1]:
import pandas as pd
from surprise import Dataset, Reader, SVD, SVDpp, NMF, KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline, BaselineOnly, NormalPredictor, SlopeOne, CoClustering
from surprise.model_selection import cross_validate



# Завантажимо дані з файлів
movies = pd.read_csv('/content/movies.dat', delimiter='::', engine='python', header=None, names=['movie_id', 'title', 'genres'], encoding='ISO-8859-1')
ratings = pd.read_csv('/content/ratings.dat', delimiter='::', engine='python', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], encoding='ISO-8859-1')
users = pd.read_csv('/content/users.dat', delimiter='::', engine='python', header=None, names=['user_id', 'gender', 'age', 'occupation', 'zip'], encoding='ISO-8859-1')

# Виведемо кілька перших рядків, щоб перевірити завантаження
print(movies.head())
print(ratings.head())
print(users.head())


  error: subprocess-exited-with-error
  
  × Building wheel for scikit-surprise (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [117 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-312
      creating build\lib.win-amd64-cpython-312\surprise
      copying surprise\accuracy.py -> build\lib.win-amd64-cpython-312\surprise
      copying surprise\builtin_datasets.py -> build\lib.win-amd64-cpython-312\surprise
      copying surprise\dataset.py -> build\lib.win-amd64-cpython-312\surprise
      copying surprise\dump.py -> build\lib.win-amd64-cpython-312\surprise
      copying surprise\reader.py -> build\lib.win-amd64-cpython-312\surprise
      copying surprise\trainset.py -> build\lib.win-amd64-cpython-312\surprise
      copying surprise\utils.py -> build\lib.win-amd64-cpython-312\surprise
      copying surprise\__init__.py -> build\lib.win-amd64-cpython-312\surprise
      cop

  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build scikit-surprise


ModuleNotFoundError: No module named 'surprise'

In [19]:
# Створимо DataFrame для Surprise
data = pd.merge(ratings, movies, on='movie_id')

# Виберемо тільки 100 користувачів та 1000 фільмів для прикладу
small_data = data[data['user_id'].isin(data['user_id'].unique()[:100]) & data['movie_id'].isin(data['movie_id'].unique()[:1000])]

reader = Reader(rating_scale=(1, 5))
dataset = Dataset.load_from_df(small_data[['user_id', 'movie_id', 'rating']], reader)


# Перевіримо перші кілька рядків зведеного DataFrame
print(small_data[['user_id', 'movie_id', 'rating']].head())

   user_id  movie_id  rating
0        1    1193.0     5.0
1        2    1193.0     5.0
2       12    1193.0     4.0
3       15    1193.0     4.0
4       17    1193.0     5.0


### Оцінка алгоритмів SVD та SVDpp

In [20]:
def evaluate_models(dataset):
    algorithms = [
        ('SVD', SVD()),
        ('SVDpp', SVDpp())
    ]

    results = []
    for name, algorithm in algorithms:
        print(f"Evaluating {name}...")
        result = cross_validate(algorithm, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True)
        avg_rmse = result['test_rmse'].mean()
        avg_mae = result['test_mae'].mean()
        results.append((name, avg_rmse, avg_mae))

    return results

results = evaluate_models(dataset)

# Виведемо результати
for name, rmse, mae in results:
    print(f"Algorithm: {name}, RMSE: {rmse:.4f}, MAE: {mae:.4f}")

Evaluating SVD...
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9096  0.9350  0.9408  0.9279  0.9201  0.9267  0.0110  
MAE (testset)     0.7229  0.7438  0.7392  0.7411  0.7266  0.7347  0.0084  
Fit time          0.24    0.22    0.22    0.21    0.24    0.22    0.01    
Test time         0.02    0.02    0.03    0.02    0.02    0.02    0.00    
Evaluating SVDpp...
Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9114  0.9106  0.9191  0.9176  0.9209  0.9159  0.0041  
MAE (testset)     0.7184  0.7210  0.7280  0.7271  0.7243  0.7238  0.0036  
Fit time          5.79    4.89    5.36    4.88    5.32    5.25    0.34    
Test time         1.15    0.84    1.33    0.87    1.45    1.13    0.24    
Algorithm: SVD, RMSE: 0.9267, MAE: 0.7347
Algorithm: SVDpp, RMSE: 0.9159, MAE: 0.7238


In [21]:

def evaluate_models(dataset):
    algorithms = [
        ('SVD', SVD()),
        ('SVDpp', SVDpp()),
        ('NMF', NMF())
    ]

    results = []
    for name, algorithm in algorithms:
        print(f"Evaluating {name}...")
        result = cross_validate(algorithm, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True)
        avg_rmse = result['test_rmse'].mean()
        avg_mae = result['test_mae'].mean()
        results.append((name, avg_rmse, avg_mae))

    return results

results = evaluate_models(dataset)

# Виведемо результати
for name, rmse, mae in results:
    print(f"Algorithm: {name}, RMSE: {rmse:.4f}, MAE: {mae:.4f}")


Evaluating SVD...
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9375  0.9295  0.9151  0.9103  0.9351  0.9255  0.0109  
MAE (testset)     0.7400  0.7401  0.7287  0.7169  0.7353  0.7322  0.0087  
Fit time          0.40    0.39    0.34    0.34    0.32    0.36    0.03    
Test time         0.03    0.04    0.03    0.04    0.03    0.03    0.01    
Evaluating SVDpp...
Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9143  0.9172  0.9095  0.9140  0.9192  0.9148  0.0033  
MAE (testset)     0.7257  0.7300  0.7228  0.7173  0.7246  0.7241  0.0041  
Fit time          5.11    5.19    5.09    4.94    5.36    5.14    0.14    
Test time         0.82    1.29    0.84    1.30    0.94    1.04    0.21    
Evaluating NMF...
Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold


На основі отриманих результатів можемо зробити кілька висновків про продуктивність алгоритмів рекомендаційних систем SVD, SVDpp та NMF:

###1. RMSE (Root Mean Square Error) та MAE (Mean Absolute Error):

- SVD: RMSE = 0.9255, MAE = 0.7322
- SVDpp: RMSE = 0.9148, MAE = 0.7241
- NMF: RMSE = 0.9827, MAE = 0.7723

###2. Точність моделей:

- SVDpp показав найнижчий RMSE і MAE серед трьох алгоритмів, що вказує на його вищу точність у порівнянні з SVD та NMF.
- SVD показав середні результати, поступаючись SVDpp, але перевершуючи NMF.
- NMF мав найвищі значення RMSE і MAE, що вказує на нижчу точність у порівнянні з SVD та SVDpp.

### 3. Час виконання:

- SVDpp має значно довший час підготовки моделі (фітингу) в порівнянні з SVD та NMF. Це слід враховувати, якщо важливий час обробки даних.
- NMF показав швидший час фітингу, але результативність його нижча за SVD та SVDpp.

##Висновки:
- Точність: SVDpp є найточнішим алгоритмом серед розглянутих. Якщо ваша основна мета - точність прогнозування, SVDpp є кращим вибором.
- Час обчислення: SVD має баланс між точністю і часом виконання. Якщо важливо зменшити час обчислення без значної втрати точності, SVD може бути оптимальним вибором.
- Нижча продуктивність: NMF показав найгірші результати точності, але може бути корисним для специфічних сценаріїв, де його властивості можуть бути перевагою, або коли потрібно швидко отримати результати.

In [22]:
def evaluate_models(dataset):
    algorithms = [
        ('SVD', SVD()),
        ('SVDpp', SVDpp()),
        ('NMF', NMF()),
        ('KNNBasic', KNNBasic()),
        ('KNNWithMeans', KNNWithMeans()),
        ('KNNWithZScore', KNNWithZScore()),
        ('KNNBaseline', KNNBaseline()),
        ('BaselineOnly', BaselineOnly()),
        ('NormalPredictor', NormalPredictor()),
        ('SlopeOne', SlopeOne()),
        ('CoClustering', CoClustering())
    ]

    results = []
    for name, algorithm in algorithms:
        print(f"Evaluating {name}...")
        result = cross_validate(algorithm, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True)
        avg_rmse = result['test_rmse'].mean()
        avg_mae = result['test_mae'].mean()
        results.append((name, avg_rmse, avg_mae))

    return results

results = evaluate_models(dataset)

# Виведемо результати
for name, rmse, mae in results:
    print(f"Algorithm: {name}, RMSE: {rmse:.4f}, MAE: {mae:.4f}")

Evaluating SVD...
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9288  0.9140  0.9188  0.9325  0.9485  0.9285  0.0120  
MAE (testset)     0.7342  0.7207  0.7255  0.7470  0.7494  0.7353  0.0114  
Fit time          0.27    0.26    0.24    0.22    0.22    0.24    0.02    
Test time         0.02    0.02    0.02    0.02    0.02    0.02    0.00    
Evaluating SVDpp...
Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9137  0.9246  0.9188  0.9274  0.9133  0.9196  0.0057  
MAE (testset)     0.7188  0.7312  0.7245  0.7274  0.7212  0.7246  0.0044  
Fit time          5.61    4.90    5.33    5.40    5.49    5.34    0.24    
Test time         1.45    0.95    1.42    0.87    1.41    1.22    0.26    
Evaluating NMF...
Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold

##Найменший RMSE (корінь середньоквадратичної помилки):

- SVDpp показав найнижчий RMSE 0.9196, недалеко від нього є BaselineOnly з RMSE 0.9181.

##Найменший MAE (середня абсолютна помилка):

- BaselineOnly показав найнижчий MAE 0.7293, трохи попереду SVDpp з MAE 0.7246.

##Порівняння ефективності:

- SVDpp показує стабільно добрі результати як у RMSE, так і у MAE, що свідчить про його надійність на різних розбиттях даних.
- BaselineOnly показує конкурентоспроможні результати, особливо у MAE, що може свідчити про його ефективність як простого, але ефективного базового рішення.

##Інші розгляди:

- Алгоритми, такі як NormalPredictor, мають значно вищі помилки, що підказує, що вони можуть не бути підходящими для точних прогнозів, але можуть служити базовими для порівняння.